<center>
<h1><b>Revealing the Mechanism of WeChat Red Envelopes<b>
</center>

<center>
 Lei Qian 2024533134
</center>
<center>
 Jingqi Xu 2024533083
</center>
<center>
 Zihao Zhang 2024533113
</center>

<center>
date:2025/12/08 
</center>

### **1.Observation**

## **2.Visualization**

## **3.Distribution**

## **4.Model Testing**

## **5.Generative Modeling**

## **6.From Modeling to Decision-Making Policy**

## **7.Further Explorations**